---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [18]:
from collections import defaultdict
import networkx
import pandas

# This line must be commented out when submitting to the autograder
#!head email_network.txt

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [19]:
email_network = pandas.read_table('email_network.txt', dtype={"#Sender": str, "Recipient": str})

In [3]:
def answer_one():
    graph = networkx.MultiDiGraph()
    tuples = [(sender, recipient, {"time": time})
              for (sender, recipient, time) in email_network.values]
    graph.add_edges_from(tuples)
    return graph

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [4]:
def answer_two():
    one = answer_one()
    return (one.order(), one.size())

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [5]:
def answer_three():
    emails = answer_one()
    nodes = emails.nodes()
    other_nodes = len(nodes) - 1
    fully_connected = all((len(emails.neighbors(node)) == other_nodes for node in nodes))
    undirected = emails.to_undirected()
    all_connected = True
    for left_node in nodes:
        for right_node in nodes:
            if left_node != right_node and not undirected.has_edge(left_node, right_node):
                all_connected = False
                break
        if not all_connected:
            break
    return networkx.is_strongly_connected(emails), networkx.is_weakly_connected(emails)
    # return fully_connected, all_connected

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [ ]:
def answer_four():
    one = answer_one()
    return len(max(networkx.weakly_connected_component_subgraphs(one), key=len).nodes())        

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [7]:
def answer_five():
    return len(max(networkx.strongly_connected_components(one), key=len))

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [8]:
def answer_six():
    one = answer_one()
    return max(networkx.algorithms.strongly_connected_component_subgraphs(one),
               key=len)

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [9]:
def answer_seven():
    return networkx.average_shortest_path_length(strongly_connected)

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [10]:
def answer_eight():
    return networkx.diameter(strongly_connected)

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [11]:
def answer_nine():
    return networkx.periphery(strongly_connected)

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [12]:
def answer_ten():
    return networkx.center(strongly_connected)

### Question 11

Which node in G_sc has the most shortest paths to other nodes whose distance equal the diameter of G_sc?


How many of these paths are there?


*This function should return a tuple (name of node, number of paths).*

In [13]:
def answer_eleven():
    candidates = networkx.periphery(strongly_connected)
    outcomes = defaultdict(int)
    for candidate_1 in candidates:
        for candidate_2 in candidates:
            if (candidate_1 != candidate_2
                and networkx.has_path(strongly_connected,
                                      candidate_1,
                                      candidate_2)):
                outcomes[candidate_1] += len(list(networkx.all_shortest_paths(strongly_connected,
                                                                              candidate_1,
                                                                              candidate_2)))
    node = max(outcomes, key=lambda key: outcomes[key])
    return node, outcomes[node]

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [14]:
def answer_twelve():
    node, count = answer_eleven()    
    center = networkx.center(strongly_connected)
    remove = [networkx.node_connectivity(strongly_connected, node, other) for other in center]
    return max(remove)

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [15]:
def answer_thirteen():
    return strongly_connected.to_undirected()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [16]:
def answer_fourteen():
    graph = answer_thirteen()
    graph = networkx.Graph(graph)
    return networkx.transitivity(graph), networkx.average_clustering(graph)